# Capstone Project - The Battle of Neighborhood

## Opening a new restaurant in Manhattan

#### Import libraries

In [1]:
import numpy as np
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json

!conda install -c conda-forge geopy --yes 

from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize 
!conda install -c conda-forge beautifulsoup4 --yes
from bs4 import BeautifulSoup
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium
 
!conda install -c conda-forge lxml --yes
!conda install -c conda-forge html5lib --yes

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


#### Download and explore the data

In [8]:
!pip install wget
import wget
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json'
data = wget.download(url)

In [9]:
data

'newyork_data.json'

In [11]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [12]:
newyork_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

#### Create a list of the neighborhoods. 

In [13]:
neighborhoods_data = newyork_data['features']

In [14]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

#### Transform the data into a pandas dataframe

In [15]:
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

neighborhoods = pd.DataFrame(columns=column_names)

In [16]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [17]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [18]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


#### Use geopy library to get the longitude and latitude values

In [19]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="hello_new_york")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


#### Create a map of New york with neighborhoods

In [20]:
map_Newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Newyork)  
    
map_Newyork

#### Create a new dataframe of Manhattan data

In [21]:
Manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
Manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


#### Use geopy library to get the longitude and latitude values of Manhattan

In [22]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="hello_manhattan")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


#### Create a map of Manhattan with neighborhoods

In [25]:
map_Manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, label in zip(Manhattan_data['Latitude'], Manhattan_data['Longitude'], Manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Manhattan)  
    
map_Manhattan

#### Define Foursquare Credentials

In [26]:
CLIENT_ID = 'BOV0B3AT33OSSA0NKOMHVI225NCDOQKU5ESOGLNCSCPHA54P' 
CLIENT_SECRET = 'QCICVP0PBMPBPC4XKDRO4ZFLZU0PAALAYLUA0TQTG0YCRMXT' 
VERSION = '20180605' 
LIMIT = 100 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BOV0B3AT33OSSA0NKOMHVI225NCDOQKU5ESOGLNCSCPHA54P
CLIENT_SECRET:QCICVP0PBMPBPC4XKDRO4ZFLZU0PAALAYLUA0TQTG0YCRMXT


#### What is the first neighborhood

In [27]:
Manhattan_data.loc[0, 'Neighborhood']

'Marble Hill'

#### Get the neighborhoods latitude and longitude values

In [28]:
neighborhood_latitude = Manhattan_data.loc[0, 'Latitude'] 
neighborhood_longitude = Manhattan_data.loc[0, 'Longitude'] 

neighborhood_name = Manhattan_data.loc[0, 'Neighborhood'] 

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Marble Hill are 40.87655077879964, -73.91065965862981.


#### Get the top 100 venues in Marble Hill within the radius of 500 meters

In [29]:
LIMIT = 100

radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
CLIENT_ID,
CLIENT_SECRET,
VERSION,
neighborhood_latitude,
neighborhood_longitude,
radius,
LIMIT)

url

'https://api.foursquare.com/v2/venues/explore?&client_id=BOV0B3AT33OSSA0NKOMHVI225NCDOQKU5ESOGLNCSCPHA54P&client_secret=QCICVP0PBMPBPC4XKDRO4ZFLZU0PAALAYLUA0TQTG0YCRMXT&v=20180605&ll=40.87655077879964,-73.91065965862981&radius=500&limit=100'

In [30]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fcd0e4985adf10fc1bd46f8'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Marble Hill',
  'headerFullLocation': 'Marble Hill, New York',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 22,
  'suggestedBounds': {'ne': {'lat': 40.88105078329964,
    'lng': -73.90471933917806},
   'sw': {'lat': 40.87205077429964, 'lng': -73.91659997808156}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b4429abf964a52037f225e3',
       'name': "Arturo's",
       'location': {'address': '5198 Broadway',
        'crossStreet': 'at 225th St.',
        'lat': 40.87441177110231,
        'lng': -73.91027100981574,
        'labeledLatLngs': [{'label'

#### Clean the json and stucture it into a pandas dataframe

In [31]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [32]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues)

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-32-3a23abf09381>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues)


,name,categories,lat,lng
0,Arturo's,Pizza Place,40.874412,-73.910271
1,Bikram Yoga,Yoga Studio,40.876844,-73.906204
2,Tibbett Diner,Diner,40.880404,-73.908937
3,Dunkin',Donut Shop,40.877136,-73.906666
4,Starbucks,Coffee Shop,40.877531,-73.905582


In [33]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

22 venues were returned by Foursquare.


#### Create a function that repeat the proces to all the neighborhoods in Manhattan

In [34]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
       
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Create a new dataframe 

In [35]:
Manhattan_venues = getNearbyVenues(names=Manhattan_data['Neighborhood'],
                                   latitudes=Manhattan_data['Latitude'],
                                   longitudes=Manhattan_data['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [36]:
print(Manhattan_venues.shape)

(3203, 7)


In [40]:
Manhattan_venues.groupby('Neighborhood').count()


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,66,66,66,66,66,66
Carnegie Hill,86,86,86,86,86,86
Central Harlem,45,45,45,45,45,45
Chelsea,100,100,100,100,100,100
Chinatown,100,100,100,100,100,100
Civic Center,100,100,100,100,100,100
Clinton,100,100,100,100,100,100
East Harlem,40,40,40,40,40,40
East Village,100,100,100,100,100,100


In [41]:
print('There are {} uniques categories.'.format(len(Manhattan_venues['Venue Category'].unique())))

There are 321 uniques categories.


#### Analyze each neigborhood

In [43]:
Manhattan_onehot = pd.get_dummies(Manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

Manhattan_onehot['Neighborhood'] = Manhattan_venues['Neighborhood'] 

fixed_columns = [Manhattan_onehot.columns[-1]] + list(Manhattan_onehot.columns[:-1])
Manhattan_onehot = Manhattan_onehot[fixed_columns]

Manhattan_onehot.head(10)

,Neighborhood,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Smoothie Shop,Snack Place,Soba Restaurant,Soccer Field,Social Club,Soup Place,South Indian Restaurant,Southern / Soul Food Restaurant,Sp

#### Group rows by neighborhoods

In [44]:
Manhattan_grouped = Manhattan_onehot.groupby('Neighborhood').mean().reset_index()
Manhattan_grouped

,Neighborhood,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Smoothie Shop,Snack Place,Soba Restaurant,Soccer Field,Social Club,Soup Place,South Indian Restaurant,Southern / Soul Food Restaurant,Sp

#### Let's find out how many categories can be curated from all the returned venues

In [46]:
print('There are {} uniques categories.'.format(len(Manhattan_venues['Venue Category'].unique())))

There are 321 uniques categories.


In [48]:
Manhattan_venues['Venue Category'].unique()[:50]

array(['Pizza Place', 'Yoga Studio', 'Diner', 'Donut Shop', 'Coffee Shop',
       'Gym', 'Pharmacy', 'Department Store', 'Seafood Restaurant',
       'Tennis Stadium', 'Supplement Shop', 'Discount Store',
       'Ice Cream Shop', 'Sandwich Place', 'Video Game Store',
       'Steakhouse', 'Kids Store', 'Deli / Bodega', 'Hotel',
       'Greek Restaurant', 'Cocktail Bar', 'Chinese Restaurant', 'Bakery',
       'Spa', 'Noodle House', 'Bike Shop', 'New American Restaurant',
       'Dessert Shop', 'Salon / Barbershop', 'Bubble Tea Shop',
       'Indie Movie Theater', 'American Restaurant', 'Tea Room',
       'Hotpot Restaurant', 'Roof Deck', 'Asian Restaurant',
       'Spanish Restaurant', 'Museum', 'Garden Center', 'Historic Site',
       'Thai Restaurant', 'Furniture / Home Store', 'Boutique',
       'Shanghai Restaurant', 'Music Venue', 'Malay Restaurant',
       'Organic Grocery', 'Cosmetics Shop', 'Supermarket',
       'Vietnamese Restaurant'], dtype=object)

In [49]:
"Restaurant" in Manhattan_venues['Venue Category'].unique()

True

#### Create a new dataframe for Restaurant data only

In [51]:
Manhattan_restaurants = Manhattan_grouped[["Neighborhood","Restaurant"]]

In [52]:
Manhattan_restaurants.head()

,Neighborhood,Restaurant
0,Battery Park City,0.000000
1,Carnegie Hill,0.011628
2,Central Harlem,0.000000
3,Chelsea,0.000000
4,Chinatown,0.000000


In [53]:
Manhattan_restaurants.shape

(40, 2)

#### Cluster neighborhoods

In [61]:
kclusters = 5

Manhattan_grouped_clustering = Manhattan_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Manhattan_grouped_clustering)

kmeans.labels_[0:10] 

array([1, 0, 1, 0, 0, 1, 1, 3, 0, 1])

In [62]:
Manhattan_merged = Manhattan_restaurants.copy()

Manhattan_merged["Cluster Labels"] = kmeans.labels_

In [63]:
Manhattan_merged.head(10)

,Neighborhood,Restaurant,Cluster Labels
0,Battery Park City,0.000000,1
1,Carnegie Hill,0.011628,0
2,Central Harlem,0.000000,1
3,Chelsea,0.000000,0
4,Chinatown,0.000000,0
5,Civic Center,0.000000,1
6,Clinton,0.020000,1
7,East Harlem,0.025000,3
8,East Village,0.000000,0
9,Financial District,0.010000,1


#### To add latitude and longitude data

In [64]:
Manhattan_merged = Manhattan_merged.merge(Manhattan_data.set_index("Neighborhood"), on="Neighborhood")

print(Manhattan_merged.shape)
Manhattan_merged.head(10)

(40, 6)


,Neighborhood,Restaurant,Cluster Labels,Borough,Latitude,Longitude
0,Battery Park City,0.000000,1,Manhattan,40.711932,-74.016869
1,Carnegie Hill,0.011628,0,Manhattan,40.782683,-73.953256
2,Central Harlem,0.000000,1,Manhattan,40.815976,-73.943211
3,Chelsea,0.000000,0,Manhattan,40.744035,-74.003116
4,Chinatown,0.000000,0,Manhattan,40.715618,-73.994279
5,Civic Center,0.000000,1,Manhattan,40.715229,-74.005415
6,Clinton,0.020000,1,Manhattan,40.759101,-73.996119
7,East Harlem,0.025000,3,Manhattan,40.792249,-73.944182
8,East Village,0.000000,0,Manhattan,40.727847,-73.982226
9,Financial District,0.010000,1,Manhattan,40.707107,-74.010665


In [65]:
print(Manhattan_merged.shape)
Manhattan_merged.sort_values(["Cluster Labels"], inplace=True)
Manhattan_merged

(40, 6)


,Neighborhood,Restaurant,Cluster Labels,Borough,Latitude,Longitude
19,Lower East Side,0.000000,0,Manhattan,40.717807,-73.980890
37,Washington Heights,0.011905,0,Manhattan,40.851903,-73.936900
36,Upper West Side,0.020408,0,Manhattan,40.787658,-73.977059
35,Upper East Side,0.000000,0,Manhattan,40.775639,-73.960508
34,Turtle Bay,0.010000,0,Manhattan,40.752042,-73.967708
31,Sutton Place,0.000000,0,Manhattan,40.760280,-73.963556
29,Soho,0.000000,0,Manhattan,40.722184,-74.000657
27,Noho,0.000000,0,Manhattan,40.723259,-73.988434
20,Manhattan Valley,0.000000,0,Manhattan,40.797307,-73.964286
18,Little Italy,0.000000,0,Manhattan,40.719324,-73.997305


#### Visualize the results

In [66]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(Manhattan_merged['Latitude'], Manhattan_merged['Longitude'], Manhattan_merged['Neighborhood'], Manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [67]:
map_clusters.save('map_clusters.html')

#### Examine clusters

#### Cluster 1

In [74]:
Manhattan_merged.loc[Manhattan_merged['Cluster Labels'] == 0]

,Neighborhood,Restaurant,Cluster Labels,Borough,Latitude,Longitude
19,Lower East Side,0.000000,0,Manhattan,40.717807,-73.980890
37,Washington Heights,0.011905,0,Manhattan,40.851903,-73.936900
36,Upper West Side,0.020408,0,Manhattan,40.787658,-73.977059
35,Upper East Side,0.000000,0,Manhattan,40.775639,-73.960508
34,Turtle Bay,0.010000,0,Manhattan,40.752042,-73.967708
31,Sutton Place,0.000000,0,Manhattan,40.760280,-73.963556
29,Soho,0.000000,0,Manhattan,40.722184,-74.000657
27,Noho,0.000000,0,Manhattan,40.723259,-73.988434
20,Manhattan Valley,0.000000,0,Manhattan,40.797307,-73.964286
18,Little Italy,0.000000,0,Manhattan,40.719324,-73.997305


#### Cluster 2

In [73]:
Manhattan_merged.loc[Manhattan_merged['Cluster Labels'] == 1]

,Neighborhood,Restaurant,Cluster Labels,Borough,Latitude,Longitude
2,Central Harlem,0.000000,1,Manhattan,40.815976,-73.943211
32,Tribeca,0.000000,1,Manhattan,40.721522,-74.010683
5,Civic Center,0.000000,1,Manhattan,40.715229,-74.005415
6,Clinton,0.020000,1,Manhattan,40.759101,-73.996119
26,Murray Hill,0.030000,1,Manhattan,40.748303,-73.978332
23,Midtown,0.000000,1,Manhattan,40.754691,-73.981669
22,Marble Hill,0.000000,1,Manhattan,40.876551,-73.910660
38,West Village,0.000000,1,Manhattan,40.734434,-74.006180
9,Financial District,0.010000,1,Manhattan,40.707107,-74.010665
17,Lincoln Square,0.000000,1,Manhattan,40.773529,-73.985338


#### Cluster 3

In [72]:
Manhattan_merged.loc[Manhattan_merged['Cluster Labels'] == 2]

,Neighborhood,Restaurant,Cluster Labels,Borough,Latitude,Longitude
30,Stuyvesant Town,0.0,2,Manhattan,40.731,-73.974052


#### Cluster 4

In [71]:
Manhattan_merged.loc[Manhattan_merged['Cluster Labels'] == 3]

,Neighborhood,Restaurant,Cluster Labels,Borough,Latitude,Longitude
25,Morningside Heights,0.000000,3,Manhattan,40.808000,-73.963896
7,East Harlem,0.025000,3,Manhattan,40.792249,-73.944182
28,Roosevelt Island,0.038462,3,Manhattan,40.762160,-73.949168
21,Manhattanville,0.000000,3,Manhattan,40.816934,-73.957385
33,Tudor City,0.024691,3,Manhattan,40.746917,-73.971219
15,Inwood,0.053571,3,Manhattan,40.867684,-73.921210


#### Cluster 5

In [75]:
Manhattan_merged.loc[Manhattan_merged['Cluster Labels'] == 4]

,Neighborhood,Restaurant,Cluster Labels,Borough,Latitude,Longitude
24,Midtown South,0.0,4,Manhattan,40.74851,-73.988713


#### Results and discussion

New York City’s restaurant industry had 23,650 establishments and most of it concentrated in Manhattan.
By accessing the total number and distributions of restaurants we can see that the data trends of each distinctive boroughs/ districts for restaurants is not equally distributed for Manhattan. 
The analyzed data obtained via exploratory data analysis has given us the findings that the total numbers of restaurants in Manhattan is significantly outweighs Lower Manhattan, Hell's Kitchen, Upper East Side or Upper West Side then Midtown Manhattan or Peter Cooper Village.


#### Conclusion


Most of restaurants are concentrated in the lower and central area of Manhattan, in the cluster 0 and cluster 1. On the other hand, cluster 2 and cluster 4 has few restaurants at all. This represents a great opportunity and high potential areas to open new restaurants anywhere in cluster 2 and 4, as there is little competition from existing restaurants. Meanwhile, restaurants in cluster 0 and cluster 1 are likely suffering from intense competition due to oversupply and high concentration of restaurants. From another perspective, this also shows that the oversupply of restaurants mostly happened in the central area of the city. Therefore, this project recommends property developers to capitalize on these findings to open new restaurants in neighborhoods in cluster 2 or cluster 4 with little competition.